# Traffic prediction model

## Abstract

This notebook explores a method to predicting traffic patterns on freeways. A naive approach to predict traffic for a particular station would be to use traffic state from all other station on a freeway. This would prove to be computatinally heavy and not feasable for a notebook. However based on this theme there are many simpler permutation of station combinations that can be fed into the model. For example; State of all stations. State of all stations in a freeway. State of all preceding stations. State of preceding staiton. State of all postceding station. State of postceding station. State of post and precdeding station, etc.

For this model the assumtion is made that cars do appear or disapear on any segment of freeway between two stations, execpt when there are intermediate onramps or offramps between stations. With this assumption, a prediction should be possible based on the preceding stations flow and occupancy. This is the model described below. 

This method should capture the intial traffic formation. This method may not capture the observed traffic propigation. Specifically backwards traveling waves of traffic. Further refinement of the model may include adding the state of postceding stations. 

This model consists of two submodels. The first model called the Station Traffic Model (STM) predicts traffic patterns for an individual Station Model (SM) based on the preceding station's flow, occupancy, and intermediate on ramp and off ramp flows. The second model called the Freeway Traffic Model (FTM) predicts traffic patterns for a freeway from the SMs, and previous station state. The model is intialized with initial station state. The model is used by feeding in previously predicted station state and pumping the first staiton flow and occupancy, and on-ramps and off-ramp flows.

Once traffic has been predicted use the fundumental diagram  based model get a predction of confidence of a stations traffic state for each time step in the FTM


## Station Models (SM)
* stations are in order of flow
* X<sub>n,t</sub>
    * X is a station flow and occupancy
    * where n is 0..N, N=number of stations
    * where t is 0..T, T=5 minute interval 
* O<sub>n-1..n,t-1</sub>
    * O is on-ramps flow
    * where n-1..n is between X<sub>n-1,t-1</sub> and X<sub>n,t-1</sub>
* F<sub>n-1..n,t-1</sub> 
    * F is off-ramps flow
    * where n-1..n is between X<sub>n-1,t-1</sub> and X<sub>n,t-1</sub>

## Station Traffic Model (STM)
#### STM<sub>n</sub>(X<sub>n-1,t-1</sub>, O<sub>n-1..n,t-1</sub>, F<sub>n-1..n,t-1</sub> ) = X'<sub>n,t</sub>
* X<sub>n-1,t-1</sub> is flow and occupancy of X<sub>n-1,t-1</sub>
* O<sub>n-1..n,t-1</sub> is flow of all OR between X<sub>n-1,t-1</sub> and X<sub>n,t</sub>
* F<sub>n-1..n,t-1</sub> is flow of all FR between X<sub>n-1,t-1</sub> and X<sub>n,t</sub>
* X'<sub>n,t</sub> is prediction of flow and occupancy for X'
    
## Freeway Traffic Model (FTM)
### Initialization at t=0 : FTM<sub>0</sub>
#### FTM<sub>0</sub>(STM<sub>1..N</sub>,  X<sub>0..N,0</sub>, O<sub>0..N,0</sub>, F<sub>0..N,0</sub>) = X'<sub>1..N,1</sub>

* X<sub>0..N,0</sub> is initial station state
* STM<sub>1..N</sub> is trained station traffic model
* O<sub>0..N,0</sub> is initial on ramp flow between stations
* F<sub>0..N,0</sub> is initial off ramp flow between stations
* X'<sub>0..N,1</sub> is predicted traffic for X<sub>0..N</sub>

### FTM at t=1..T : FTM<sub>1..T</sub>

#### FTM<sub>t</sub>(STM<sub>1..N</sub>,  X'<sub>0,p;1..N,t-1</sub>, O<sub>0..N,p</sub>, F<sub>0..N,p</sub>) = X'<sub>1..N,t</sub>
* X'<sub>1..N,t-1</sub> is previous predicted station state for station 1..0
* X'<sub>0,p</sub> is pumped station state for station 0
* STM<sub>0..N</sub> is trained station traffic model
* O<sub>0..N,p</sub> is pumped on-ramp flow between stations
* F<sub>0..N,p</sub> is pumped off-ramp flow between stations
* X'<sub>0..N,t+1</sub> is predicted traffic for X'<sub>0..N</sub>


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from os import listdir
from os.path import isfile, join
from datetime import time

import glob


import sys
sys.path.append('../')
import trafficpassion.AnalyzeWiggles as aw

In [2]:
#data locations
data_5min_path = "../data/station_5min/2015/d11/"
meta_path = "../data/station_5min/2015/meta_data/d11/"

# Get data file names

In [3]:
#get all files to process
onlyfiles = [f for f in listdir(data_5min_path) if isfile(join(data_5min_path, f))]

In [4]:
onlyfiles[0:3]

['d11_text_station_5min_2015_01_01.txt.gz',
 'd11_text_station_5min_2015_01_02.txt.gz',
 'd11_text_station_5min_2015_01_03.txt.gz']

In [5]:
colnames = [
    'Timestamp','Station','District','Freeway','Direction_of_Travel',
    'LaneType','StationLength','Samples',
    'Perc_Observed','TotalFlow','AvgOccupancy','AvgSpeed',
    'Lane1_Samples','Lane1_Flow','Lane1_AvgOcc','Lane1_AvgSpeed','Lane1_Observed',
    'Lane2_Samples','Lane2_Flow','Lane2_AvgOcc','Lane2_AvgSpeed','Lane2_Observed',
    'Lane3_Samples','Lane3_Flow','Lane3_AvgOcc','Lane3_AvgSpeed','Lane3_Observed',
    'Lane4_Samples','Lane4_Flow','Lane4_AvgOcc','Lane4_AvgSpeed','Lane4_Observed',
    'Lane5_Samples','Lane5_Flow','Lane5_AvgOcc','Lane5_AvgSpeed','Lane5_Observed',
    'Lane6_Samples','Lane6_Flow','Lane6_AvgOcc','Lane6_AvgSpeed','Lane6_Observed',
    'Lane7_Samples','Lane7_Flow','Lane7_AvgOcc','Lane7_AvgSpeed','Lane7_Observed',
    'Lane8_Samples','Lane8_Flow','Lane8_AvgOcc','Lane8_AvgSpeed','Lane8_Observed'
]
colnames = [c.lower() for c in colnames]

In [6]:
len(colnames)

52

In [7]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *

# Make spark schema

struct list was generated with the following code after reading the files with inferschema = true, then hand modified 
```
'[' + ','.join(['StructField("%s",%s(),True)'% (colnames[idx], str(i.dataType))
for idx, i in enumerate(rdd.schema)]) + ']'
```

In [8]:
#print '[\n    ' + ",\n    ".join(['StructField("%s",%s(),True)'% (colnames[idx], str(i.dataType))
#for idx, i in enumerate(rdd.schema)]) + '\n]'

# Build dataframe with spark

In [9]:
struct_list = [
    StructField("timestamp",TimestampType(),True),
    StructField("station",IntegerType(),True),
    StructField("district",IntegerType(),True),
    StructField("freeway",IntegerType(),True),
    StructField("direction_of_travel",StringType(),True),
    StructField("lanetype",StringType(),True),
    StructField("stationlength",DoubleType(),True),
    StructField("samples",IntegerType(),True),
    StructField("perc_observed",IntegerType(),True),
    StructField("totalflow",IntegerType(),True),
    StructField("avgoccupancy",DoubleType(),True),
    StructField("avgspeed",DoubleType(),True),
    StructField("lane1_samples",IntegerType(),True),
    StructField("lane1_flow",IntegerType(),True),
    StructField("lane1_avgocc",DoubleType(),True),
    StructField("lane1_avgspeed",DoubleType(),True),
    StructField("lane1_observed",IntegerType(),True),
    StructField("lane2_samples",IntegerType(),True),
    StructField("lane2_flow",IntegerType(),True),
    StructField("lane2_avgocc",DoubleType(),True),
    StructField("lane2_avgspeed",DoubleType(),True),
    StructField("lane2_observed",IntegerType(),True),
    StructField("lane3_samples",IntegerType(),True),
    StructField("lane3_flow",IntegerType(),True),
    StructField("lane3_avgocc",DoubleType(),True),
    StructField("lane3_avgspeed",DoubleType(),True),
    StructField("lane3_observed",IntegerType(),True),
    StructField("lane4_samples",IntegerType(),True),
    StructField("lane4_flow",IntegerType(),True),
    StructField("lane4_avgocc",DoubleType(),True),
    StructField("lane4_avgspeed",DoubleType(),True),
    StructField("lane4_observed",IntegerType(),True),
    StructField("lane5_samples",IntegerType(),True),
    StructField("lane5_flow",IntegerType(),True),
    StructField("lane5_avgocc",DoubleType(),True),
    StructField("lane5_avgspeed",DoubleType(),True),
    StructField("lane5_observed",IntegerType(),True),
    StructField("lane6_samples",IntegerType(),True),
    StructField("lane6_flow",IntegerType(),True),
    StructField("lane6_avgocc",DoubleType(),True),
    StructField("lane6_avgspeed",DoubleType(),True),
    StructField("lane6_observed",IntegerType(),True),
    StructField("lane7_samples",IntegerType(),True),
    StructField("lane7_flow",IntegerType(),True),
    StructField("lane7_avgocc",DoubleType(),True),
    StructField("lane7_avgspeed",DoubleType(),True),
    StructField("lane7_observed",IntegerType(),True),
    StructField("lane8_samples",IntegerType(),True),
    StructField("lane8_flow",IntegerType(),True),
    StructField("lane8_avgocc",DoubleType(),True),
    StructField("lane8_avgspeed",DoubleType(),True),
    StructField("lane8_observed",IntegerType(),True)
]

schema_struct = StructType(struct_list)

In [10]:
#node this is only the first 5 days of files for now
files = [data_5min_path + filename for filename in onlyfiles[:5]]

rdd = spark.read.csv(
    files, 
    header='false',
    timestampFormat='MM/dd/yyyy HH:mm:ss',
    schema=schema_struct,
    inferSchema='false'
)
    
rdd.take(1)

[Row(timestamp=datetime.datetime(2015, 1, 5, 0, 0), station=1100313, district=11, freeway=5, direction_of_travel=u'N', lanetype=u'FR', stationlength=None, samples=10, perc_observed=100, totalflow=6, avgoccupancy=None, avgspeed=None, lane1_samples=10, lane1_flow=6, lane1_avgocc=None, lane1_avgspeed=None, lane1_observed=1, lane2_samples=None, lane2_flow=None, lane2_avgocc=None, lane2_avgspeed=None, lane2_observed=0, lane3_samples=None, lane3_flow=None, lane3_avgocc=None, lane3_avgspeed=None, lane3_observed=0, lane4_samples=None, lane4_flow=None, lane4_avgocc=None, lane4_avgspeed=None, lane4_observed=0, lane5_samples=None, lane5_flow=None, lane5_avgocc=None, lane5_avgspeed=None, lane5_observed=0, lane6_samples=None, lane6_flow=None, lane6_avgocc=None, lane6_avgspeed=None, lane6_observed=0, lane7_samples=None, lane7_flow=None, lane7_avgocc=None, lane7_avgspeed=None, lane7_observed=0, lane8_samples=None, lane8_flow=None, lane8_avgocc=None, lane8_avgspeed=None, lane8_observed=0)]

In [11]:
rdd.count()

2129760

In [15]:
df = rdd.select('timestamp','station','totalflow','avgoccupancy').toPandas()

In [16]:
df.head()

,timestamp,station,totalflow,avgoccupancy
0,2015-01-05,1100313,6.0,NaN
1,2015-01-05,1100323,NaN,NaN
2,2015-01-05,1100326,7.0,NaN
3,2015-01-05,1100330,1.0,NaN
4,2015-01-05,1100333,10.0,NaN


# now lets group by station, time

In [ ]:
from pyspark.sql.functions import hour, mean,minute, stddev, count,max as psmax,min as psmin

In [ ]:
station_time = (
    rdd.groupBy([
        'station',
        hour("timestamp").alias("hour"),
        minute("timestamp").alias("minute")
    ]).agg(
        mean("totalflow").alias("flow_mean"),
        stddev("totalflow").alias("flow_std"),
        count("totalflow").alias("flow_count"),
        psmax("totalflow").alias("flow_max"),
        psmin("totalflow").alias("flow_min")
    )
)
#station_time.show(10)

In [ ]:
df = station_time.toPandas()

In [ ]:
df.station.unique().shape

In [ ]:
df['flow_std_plus_mean'] = df.flow_mean + df.flow_std
df['flow_std_minus_mean'] = df.flow_mean - df.flow_std

In [ ]:
df['time'] = df.apply(lambda x:time(int(x.hour),int(x.minute)),axis = 1)

In [ ]:
df.sort_values('time',inplace=True)

In [ ]:
for g,g_df in  df.groupby('station'):
    g_df.plot(
        x='time', 
        y=['flow_mean','flow_std_plus_mean','flow_std_minus_mean'],
        title='stat plots for %i'%g,
        rot=90
    )
    plt.show()
    break

In [ ]:

from scipy.interpolate import interp1d, Akima1DInterpolator
from sklearn import preprocessing
def interpolate(meanVector, kind, factor):
    y = meanVector
    y_len = len(y)
    x = np.arange(0,y_len)
    
    interpolator = {
        'akima': Akima1DInterpolator(x, y),
        'cubic': interp1d(x, y, kind='cubic'),
        'linear': interp1d(x, y, kind='linear')
    }
    
    interpolate = interpolator[kind]

    mid_factor = factor/2
    
    interp = [interpolate(np.arange(i,y_len, factor)) for i in range(factor)]
    myArray = reduce(lambda x,y:x+y,interp)

    my_x = np.arange(mid_factor,y_len, factor)
    
    extrapolator = {
        'akima': Akima1DInterpolator(my_x, myArray/factor),
        'cubic': interp1d(my_x, myArray/factor, kind='cubic'),
        'linear': interp1d(my_x, myArray/factor, kind='linear')
    }
    
    extrapolate = extrapolator[kind]
    
    new_x = np.arange(mid_factor,y_len-mid_factor)
    interpolated = extrapolate( np.arange(mid_factor,y_len-mid_factor))

    #wut??
    #pad front and back with mean vector 
    xprime = np.append(np.arange(0,mid_factor), new_x)
    xprime = np.append(xprime, np.arange(max(xprime)+1,y_len))
    yprime = np.append(y[:mid_factor], interpolated)
    yprime = np.append(yprime, y[-mid_factor:])

    return yprime


def smooth_vector(meanVector, kind='akima', factor=6):
    smoothedVector = interpolate(meanVector, kind, factor )
    diff = meanVector - smoothedVector
    diffVector = diff/np.linalg.norm(diff) 
    
    return {  
        'smoothedVector': smoothedVector, 
        'diffVector': diffVector
    }

vectors = smooth_vector(g_df['flow_mean'].values, 'akima', 12)

In [ ]:
vectors = smooth_vector(g_df['flow_mean'].values, 'akima', 12)
g_df['smoothedVector'] = vectors['smoothedVector']
g_df['diffVector'] = vectors['diffVector']

g_df.plot(
    x='time',
    y=['flow_mean','smoothedVector'], 
    rot=90,
    title='mean vector plot for %i'%(g)
)

g_df.plot(
    x='time',
    y='diffVector',
    rot=90,
    linewidth=2,
    title='diffVector plot for %i'%(g)
)




# Get metadata and identify all FR OR and their preceding and antceding station

In [ ]:
def loadMeta():
    meta_dir='../data/External/meta/2015/d11/d11_text_meta_2015_*.txt'
    meta_files = glob.glob(meta_dir)

    meta_file_list = []
    for meta_file in meta_files:
        date = str('_'.join(meta_file.split('_')[4:7])).split('.')[0]
        df = pd.read_table(meta_file, index_col=None, header=0)
        date_col = pd.Series([date] * len(df))
        df['file_date'] = date_col
        # drop rows that are missing latitude / longitude values
        #df.dropna(inplace=True, subset=['Latitude', 'Longitude'], how='any')
        meta_file_list.append(df)

    meta_frame = pd.concat(meta_file_list).drop_duplicates(subset='ID', keep='last')

    usefwy = [ 56, 125, 805,  52, 163,   8,  15,   5, 905,  78,  94,  54]

    meta_frame = meta_frame[meta_frame.Fwy.apply(lambda x: x in usefwy)]

    #Add freeway name FwyDir
    meta_frame['freeway'] = meta_frame.Fwy.apply(str) + meta_frame.Dir
    
    r_c = {}
    for c in meta_frame.columns:
        r_c[c]=c.lower()
    
    meta_frame=meta_frame.rename(columns = r_c )
    return meta_frame

loadMeta().head(1)

In [ ]:
loadMeta().head(1)

In [ ]:
#loadMeta.type.apply(lambda x: x in [])

In [ ]:
station_type_dict = loadMeta().loc[:,['id','type']].set_index('id').to_dict()['type']

station_df = df.groupby('station')

for fwy, fwy_df in loadMeta().groupby('fwy'):
    for d, d_df in fwy_df.groupby('dir'):
        d_df.sort_values('abs_pm', inplace=True)
        d_df.reset_index(drop=True, inplace=True)
        
        ramps_index = d_df[d_df.type.apply(lambda x: x in ['FR','OR'])].index.tolist()
        
        b_i = [max(i-1,min(ramps_index)) for i in ramps_index]
        b_j = [min(i+1,max(ramps_index)) for i in ramps_index]
        
        ramps_i = d_df.iloc[b_i,:].id
        ramps_j = d_df.iloc[b_j,:].id
        ramps = d_df.iloc[ramps_index,:].id
                
        for p in zip(ramps_i,ramps,ramps_j):
            fig, axs = plt.subplots(figsize=[15,8], ncols=2, nrows=3, sharex=True)
            
            for idx, r in enumerate(p):                
                r_df = station_df.get_group(r)
                vectors = smooth_vector(r_df['flow_mean'].values, 'akima', 12)
                r_df['smoothedVector'] = vectors['smoothedVector']
                r_df['diffVector'] = vectors['diffVector']

                r_df.plot(
                    x='time',
                    y=['flow_mean','smoothedVector'], 
                    rot=90,
                    title='mean vector plot for %s %s %i %s'%(fwy,d,r,station_type_dict[r]),
                    ax=axs[idx][0]
                )

                r_df.plot(
                    x='time',
                    y='diffVector',
                    rot=90,
                    linewidth=2,
                    title='diffVector plot for %s %s %i %s'%(fwy,d,r,station_type_dict[r]),
                    ax=axs[idx][1]
                )
                
            plt.show()
                     
        break
    break
        